### python -m pip install pythonnet

In [15]:
from pathlib import Path
from enum import Enum
import json

In [16]:
data_folder = Path('./test_data/')
material_data = data_folder / 'material_data.json'
column_data = data_folder / 'column_data.json'
cross_section_data = data_folder / 'cross_section_data.json'
analytical_levels = data_folder / 'analytical_levels.json'


#start ETABS application.
myETABSObject.ApplicationStart()

#close the program.
ret = myETABSObject.ApplicationExit(False)


In [18]:
import clr
clr.AddReference("System.Runtime.InteropServices")
from System.Runtime.InteropServices import Marshal

#set the following path to the installed ETABS program directory
clr.AddReference(R'C:\Program Files\Computers and Structures\ETABS 22\ETABSv1.dll')
import ETABSv1 as etabs

#create API helper object
helper = etabs.cHelper(etabs.Helper())

try:
    myETABSObject = etabs.cOAPI(helper.GetObject("CSI.ETABS.API.ETABSObject"))
except:
    print("No running instance of the program found or failed to attach.")

#create SapModel object
SapModel = etabs.cSapModel(myETABSObject.SapModel)

N_mm_C = etabs.eUnits.N_mm_C
kN_m_C = etabs.eUnits.kN_m_C

display(myETABSObject)
display(SapModel)



# create stories

In [11]:
with open(analytical_levels, 'r') as file:
    levels = json.load(file)
level_dict = {k:v for k,v in levels}
sorted_level_dict = {k: v for k, v in sorted(level_dict.items(), key=lambda item: item[1])}
story_names = list(sorted_level_dict.keys())[1:]
story_heights = []
base_level = 0
for i,level in enumerate(sorted_level_dict.values()):
    if i == 0:
        base_level = level
        previous_level = level
        continue
    story_height = level - previous_level
    previous_level = level
    story_heights.append(story_height)

story_names
story_heights

def create_stories(SapModel,base_level:float,story_names:list[str],story_heights:list[float],unit = N_mm_C) -> tuple[int,list[str],list[float]]:
    ret = SapModel.SetPresentUnits(unit)
    stories_count = len(story_names)
    IsMasterStory = [False] * stories_count
    SimilarToStory = [''] * stories_count
    SpliceAbove = [False] * stories_count
    SpliceHeight = [0.0] * stories_count
    color = [0] * stories_count
    [ret, story_names, story_heights, IsMasterStory, SimilarToStory, SpliceAbove, SpliceHeight, color] = SapModel.Story.SetStories_2(
        base_level,
        stories_count,
        story_names,
        story_heights,
        IsMasterStory,
        SimilarToStory,
        SpliceAbove,
        SpliceHeight,
        color
    )
    return (ret, list(story_names), list(story_heights))

ret,_,_ = create_stories(SapModel,base_level,story_names,story_heights)


# create materials

In [12]:
with open(material_data, 'r') as file:
    materials = json.load(file)
materials

etabs_materials = {
    'concrete' : etabs.eMatType.Concrete,
    'steel': etabs.eMatType.Steel,
    'no_design' : etabs.eMatType.NoDesign
}


def get_etabs_concrete_material(name:str):
    return {
        'Name':name,
        'Fc':float(name[2:]),
        'IsLightweight':False,
        'FcsFactor':0.0,
        'SSType':2,
        'SSHysType':4,
        'StrainAtFc':0.0022,
        'StrainUltimate':0.005,
        'FinalSlope':-0.1,
        'FrictionAngle': 0.0,
        'DilatationalAngle' : 0.0,
        'Temp' : 0.0
    }


def get_etabs_steel_material(name:str):
    if name.upper() == 'S235':
        fy,fu,efy,efu = 235.0, 360.0, 258.5, 396.0
    elif name.upper() == 'S275':
        fy,fu,efy,efu = 275.0, 430.0, 302.5, 473.0
    elif name.upper() == 'S355':
        fy,fu,efy,efu = 355.0, 510.0, 390.5, 561.0
    elif name.upper() == 'S450':
        fy,fu,efy,efu = 440.0, 550.0, 484.0, 605.0
    elif name.upper() == 'A992FY50':
        fy,fu,efy,efu = 344.738, 448.159, 379.212, 492.975
    elif name.upper() == 'A572GR50':
        fy,fu,efy,efu = 344.738, 448.159, 379.212, 492.975
    elif name.upper() == 'A913GR50':
        fy,fu,efy,efu = 344.738, 413.685, 379.212, 455.054
    else:
        fy,fu,efy,efu = 248.211, 399.896, 372.317, 439.885

    return {
    'Name' : name,
    'Fy' : fy,
    'Fu' : fu,
    'EFy' : efy,
    'EFu' : efu,
    'SSType' : 1,
    'SSHysType' : 1,
    'StrainAtHardening' : 0.015,
    'StrainAtMaxStress' : 0.11,
    'StrainAtRupture' : 0.17,
    'FinalSlope' : -0.1,
    'Temp' : 0
    }



def add_material(SapModel, name:str,unit = N_mm_C):
    
    if name.upper().startswith('FC'):
        material_type = etabs_materials['concrete']
        fc = float(name[2:])
        E = 4700 * (fc)**0.5
        U = 0.2
        A = 0.0000099
        design_material = get_etabs_concrete_material(name)
    elif name.upper().startswith('S'):
        material_type = etabs_materials['steel']
        E = 199947.98
        U = 0.3
        A = 0.0000117
        design_material = get_etabs_steel_material(name)
    else:
        material_type = etabs_materials['no_design']
        E = 199947.98
        U = 0.3
        A = 0.0000117


    ret = SapModel.SetPresentUnits(unit)
    ret = SapModel.PropMaterial.SetMaterial(name, material_type)

    ret = SapModel.PropMaterial.SetMPIsotropic(
        name,
        E,
        U,
        A,
        Temp = 0
    )

    if material_type == etabs_materials['concrete']:
        ret = SapModel.PropMaterial.SetOConcrete_1(**design_material)
    elif material_type == etabs_materials['steel']:
        ret = SapModel.PropMaterial.SetOSteel_1(**design_material)


    ret = SapModel.SetPresentUnits(kN_m_C)
    weight = 25 if material_type == etabs_materials['concrete'] else 78.5
    ret = SapModel.PropMaterial.SetWeightAndMass(name, 1, weight)

    ret = SapModel.SetPresentUnits(unit)


_ = [add_material(SapModel,x) for x in materials]


# create cross sections

In [19]:
#define rectangular frame section property
ret = SapModel.PropFrame.SetRectangle('R1', 'FC40', 12, 12)

#define frame section property modifiers
ModValue = [1000, 0, 0, 1, 1, 1, 1, 1]
ret = SapModel.PropFrame.SetModifiers('R1', ModValue)


# finish

In [20]:
SapModel = None
myETABSObject = None